In [ ]:
from transformers import BertGenerationEncoder, BertGenerationDecoder, EncoderDecoderModel
import numpy as np
from datasets import Dataset
from transformers import BertTokenizer
from collections import defaultdict
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm

In [ ]:
import datasets
idx_intent = np.load('data/total_idx_intent.npy', allow_pickle=True).item()
idx_titles = np.load('data/total_idx_titles.npy', allow_pickle=True).item()

intents = []
titles = []
for k,v in idx_intent.items():
    intents.append(v)
    titles.append(idx_titles[k])
    
# split train, test set = 8:2
test_num = int(len(intents)*0.2)

train_intent = intents[:-test_num]
train_titles = titles[:-test_num]
test_intent = intents[-test_num:]
test_titles = titles[-test_num:]

In [ ]:
from transformers import BartTokenizerFast
tokenizer = BartTokenizerFast.from_pretrained("facebook/bart-base", do_lower_case=True)

In [ ]:
title_intent = defaultdict(list)

for i in range(len(train_intent)):
    title_intent['titles'].append(train_titles[i])
    title_intent['intents'].append(train_intent[i])
    
test_title_intent = defaultdict(list)
for i in range(len(test_intent)):
    test_title_intent['titles'].append(test_titles[i])
    test_title_intent['intents'].append(test_intent[i])
    
dataset = Dataset.from_dict(title_intent)
vali_dataset = Dataset.from_dict(test_title_intent)

In [ ]:
encoder_max_length=512
decoder_max_length=64

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
    inputs = tokenizer(batch["titles"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["intents"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

In [ ]:
batch_size = 4

train_data = dataset.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["titles", "intents"]
)

In [ ]:
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_attention_mask", "labels"],
)

In [ ]:
val_data = vali_dataset.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["titles", "intents"]
)

In [ ]:
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_attention_mask", "labels"],
)

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, 
    output_dir="/home/workshop/dataset/fkd/bertGeneration/bart/",
    logging_steps=500,
    save_steps=1000,
    eval_steps=500,
    learning_rate=0.00002,
    num_train_epochs=3,
)

In [ ]:
rouge = datasets.load_metric("rouge")
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    # print(pred_str)
    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

In [ ]:
from transformers import AutoModelForSeq2SeqLM

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model = AutoModelForSeq2SeqLM.from_pretrained("./bart/checkpoint-3000/").to(device)
tokenizer = BartTokenizerFast.from_pretrained("./bart/checkpoint-3000/")

In [ ]:
def generate_summary(batch):
    # cut off at BERT max length 512
    inputs = tokenizer(batch["titles"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_summary"] = output_str

    return batch

In [ ]:
batch_size = 4  # change to 64 for full evaluation

results = vali_dataset.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["titles"])

In [ ]:
rouge_output = rouge.compute(predictions=results["pred_summary"], references=results["intents"], rouge_types=["rouge2"])["rouge2"].mid

In [ ]:
rouge_output = rouge.compute(predictions=results["pred_summary"], references=results["intents"], rouge_types=["rouge1"])["rouge1"].mid

In [ ]:
rouge_output = rouge.compute(predictions=results["pred_summary"], references=results["intents"], rouge_types=["rougeL"])["rougeL"].mid